Getting rid of the following columns: CONTRIBUTING FACTOR VEHICLE 3, CONTRIBUTING FACTOR VEHICLE 4, CONTRIBUTING FACTOR VEHICLE 5. VEHICLE TYPE CODE 3, VEHICLE TYPE CODE 4, VEHICLE TYPE CODE 5

In [1]:
import pandas as pd

# Reload the newly uploaded dataset
file_path = 'sampled_file.csv'
data = pd.read_csv(file_path)

# Drop the specified columns from the dataset
columns_to_drop = [
    "CONTRIBUTING FACTOR VEHICLE 3",
    "CONTRIBUTING FACTOR VEHICLE 4",
    "CONTRIBUTING FACTOR VEHICLE 5",
    "VEHICLE TYPE CODE 3",
    "VEHICLE TYPE CODE 4",
    "VEHICLE TYPE CODE 5"
]
cleaned_data = data.drop(columns=columns_to_drop)

# Verify the columns have been removed
cleaned_data.head(), cleaned_data.columns

(   CRASH DATE CRASH TIME    BOROUGH  ZIP CODE   LATITUDE  LONGITUDE  \
 0  05/01/2018       7:28        NaN       NaN  40.798256  -73.82744   
 1  07/13/2018      23:30      BRONX   10460.0  40.845226  -73.88079   
 2  08/15/2020      23:59     QUEENS   11435.0  40.693295  -73.80802   
 3  11/18/2019      16:20      BRONX   10455.0  40.813377  -73.91732   
 4  11/09/2019       3:40  MANHATTAN   10022.0  40.754990  -73.96536   
 
                  LOCATION                    ON STREET NAME CROSS STREET NAME  \
 0  (40.798256, -73.82744)  BRONX WHITESTONE BRIDGE                        NaN   
 1  (40.845226, -73.88079)                               NaN               NaN   
 2  (40.693295, -73.80802)  LIBERTY AVENUE                       INWOOD STREET   
 3  (40.813377, -73.91732)                               NaN               NaN   
 4   (40.75499, -73.96536)                               NaN               NaN   
 
                             OFF STREET NAME  ...  \
 0                 

In [2]:
# Step 1: Handle Missing Data

# Impute or drop missing values for location data
# Drop rows where both latitude and longitude are missing
cleaned_data = cleaned_data.dropna(subset=["LATITUDE", "LONGITUDE"], how="all")

# Fill missing contributing factors with 'Unspecified'
cleaned_data["CONTRIBUTING FACTOR VEHICLE 1"] = cleaned_data["CONTRIBUTING FACTOR VEHICLE 1"].fillna("Unspecified")
cleaned_data["CONTRIBUTING FACTOR VEHICLE 2"] = cleaned_data["CONTRIBUTING FACTOR VEHICLE 2"].fillna("Unspecified")

# Step 2: Standardize Data Formats

# Combine CRASH DATE and CRASH TIME into a single datetime column
cleaned_data["CRASH DATETIME"] = pd.to_datetime(cleaned_data["CRASH DATE"] + " " + cleaned_data["CRASH TIME"])

# Drop the original CRASH DATE and CRASH TIME columns
cleaned_data = cleaned_data.drop(columns=["CRASH DATE", "CRASH TIME"])

# Standardize borough names to uppercase
cleaned_data["BOROUGH"] = cleaned_data["BOROUGH"].str.upper()

# Step 3: Remove Duplicates
# Ensure each row is unique based on COLLISION_ID
cleaned_data = cleaned_data.drop_duplicates(subset=["COLLISION_ID"])

# Step 4: Address Outliers
# Ensure injury and fatality numbers are non-negative
numerical_columns = [
    "NUMBER OF PERSONS INJURED",
    "NUMBER OF PERSONS KILLED",
    "NUMBER OF PEDESTRIANS INJURED",
    "NUMBER OF PEDESTRIANS KILLED",
    "NUMBER OF CYCLIST INJURED",
    "NUMBER OF CYCLIST KILLED",
    "NUMBER OF MOTORIST INJURED",
    "NUMBER OF MOTORIST KILLED"
]
cleaned_data[numerical_columns] = cleaned_data[numerical_columns].clip(lower=0)

# Step 5: Normalize Entity Representations
# Standardize street name fields (e.g., remove extra spaces)
street_columns = ["ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME"]
for col in street_columns:
    cleaned_data[col] = cleaned_data[col].str.strip().str.upper()

# Step 6: Reduce Dataset for Analysis
# Select the most recent 10,000 records based on CRASH DATETIME
cleaned_data = cleaned_data.sort_values(by="CRASH DATETIME", ascending=False).head(10000)

# Step 7: Prepare Data for Visualization
# Create derived fields for day of the week and hour of the crash
cleaned_data["DAY OF WEEK"] = cleaned_data["CRASH DATETIME"].dt.day_name()
cleaned_data["HOUR OF DAY"] = cleaned_data["CRASH DATETIME"].dt.hour

# Save the cleaned data to a new CSV file for further analysis
cleaned_data.to_csv('/content/final_data.csv', index=False)
#cleaned_data.to_csv(output_path, index=False)
print("Data cleaning complete. Shape of cleaned dataset:", cleaned_data.shape)
cleaned_data.head()



<ipython-input-2-0651fe2521bc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data["CONTRIBUTING FACTOR VEHICLE 1"] = cleaned_data["CONTRIBUTING FACTOR VEHICLE 1"].fillna("Unspecified")
<ipython-input-2-0651fe2521bc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data["CONTRIBUTING FACTOR VEHICLE 2"] = cleaned_data["CONTRIBUTING FACTOR VEHICLE 2"].fillna("Unspecified")
<ipython-input-2-0651fe2521bc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

Data cleaning complete. Shape of cleaned dataset: (9185, 24)


,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,...,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,CRASH DATETIME,DAY OF WEEK,HOUR OF DAY
6146,NaN,NaN,40.707966,-73.999750,"(40.707966, -73.99975)",FDR DRIVE,NaN,NaN,0.0,0.0,...,0.0,0.0,Unsafe Lane Changing,Unspecified,4713934.0,Sedan,Sedan,2024-03-29 10:39:00,Friday,10
6562,BRONX,10458.0,40.866730,-73.887730,"(40.86673, -73.88773)",NaN,NaN,2781 MARION AVENUE,0.0,0.0,...,0.0,0.0,Unspecified,Unspecified,4708608.0,Sedan,NaN,2024-03-10 09:46:00,Sunday,9
643,MANHATTAN,10009.0,40.730877,-73.982900,"(40.730877, -73.9829)",NaN,NaN,222 1 AVENUE,0.0,0.0,...,0.0,0.0,Driver Inattention/Distraction,Unspecified,4689818.0,Sedan,Bus,2023-12-22 21:13:00,Friday,21
4549,NaN,NaN,40.902958,-73.896416,"(40.902958, -73.896416)",BROADWAY,NaN,NaN,2.0,0.0,...,2.0,0.0,Unspecified,Unspecified,4644472.0,Sedan,Sedan,2023-07-09 22:22:00,Sunday,22
8742,NaN,NaN,40.607212,-74.076820,"(40.607212, -74.07682)",HYLAN BOULEVARD,NARROWS ROAD NORTH,NaN,0.0,0.0,...,0.0,0.0,Driver Inattention/Distraction,Unspecified,4561084.0,Station Wagon/Sport Utility Vehicle,NaN,2022-09-04 05:00:00,Sunday,5
